In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 15.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV
import time as time
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [5]:
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Untuned Catboost and Lightbm

In [12]:
model_cat = CatBoostRegressor(verbose = False).fit(X_train, y_train)

In [13]:
print(np.sqrt(mean_squared_error(np.around(model_cat.predict(X_test)),y_test)))

0.7130648907789097


In [10]:
model_lgbm = LGBMRegressor().fit(X_train, y_train)

In [11]:
np.sqrt(mean_squared_error(np.around(model_lgbm.predict(X_test)),y_test))

0.7258946734362203

Tuning CatBoost

In [ ]:
#K-fold cross validation to find optimal parameters for CatBoost regressor
start_time = time.time()
param_grid = {'max_depth': [4,6,8],
              'num_leaves': [20, 31, 40],
              'learning_rate': [0.01, 0.05, 0.1],
               'reg_lambda':[0, 10, 100],
                'n_estimators':[100, 500, 1000],
                'subsample': [0.5, 0.75, 1.0]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
optimal_params = RandomizedSearchCV(estimator=CatBoostRegressor(random_state=1, verbose=False),                                                       
                             param_distributions = param_grid, n_iter = 200,
                             verbose = 1,random_state = 1,
                             n_jobs=-1,
                             cv = cv)
optimal_params.fit(X,y)
print("Optimal parameter values =", optimal_params.best_params_)
print("Optimal cross validation R-squared = ",optimal_params.best_score_)
print("Time taken = ", round((time.time()-start_time)/60), " minutes")

In [19]:
#K-fold cross validation to find optimal parameters for CatBoost regressor
start_time = time.time()
param_grid = {'max_depth': [4, 6, 8, 10, 12],
              'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
              'reg_lambda': [0, 10, 100],
              'n_estimators': [1000, 1200, 1400],
              'subsample': [0.5, 0.6, 0.75, 1.0]}

cv = KFold(n_splits = 4, shuffle = True, random_state = 1)
optimal_params = RandomizedSearchCV(estimator = CatBoostRegressor(random_state = 1, verbose = False),                                                       
                                    param_distributions = param_grid, n_iter = 50, 
                                    scoring = 'neg_mean_squared_error',
                                    verbose = 1, random_state = 1,
                                    n_jobs = -1,
                                    cv = cv)
optimal_params.fit(X_train, y_train)

print("Best: %f using %s" % (np.sqrt(-optimal_params.best_score_), optimal_params.best_params_))
print("Time taken = ", round((time.time()-start_time)/60), " minutes")

Fitting 4 folds for each of 50 candidates, totalling 200 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
print(np.sqrt(mean_squared_error(np.around(model_cat.predict(X_test)),y_test)))